In [1]:

%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from FRVRS import nu
from pandas import DataFrame, Series
import re
import pandas as pd


# Clean up Entities from Domain Documents

Previously downloaded all documents from https://nextcentury.atlassian.net/wiki/spaces/ITMC/pages/2991849482/Domain+Documents and converted them all to PDF files and stored them in the Domain_Knowledge folder.

In [3]:

# load documents
domain_doc_ners_df = nu.load_data_frames(domain_doc_ners_df='domain_doc_ners_df')['domain_doc_ners_df']
column_descriptions_df = nu.get_column_descriptions(domain_doc_ners_df)

No pickle exists for domain_doc_ners_df - attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv.


In [4]:

print(domain_doc_ners_df.shape)
domain_doc_ners_df.sample(4).dropna(axis='columns', how='all').T

(302127, 16)


,67084,154923,298064,289437
file_path,../data/Domain_Knowledge/Fundamentals of Milit...,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Rules of Engagement/A...,../data/Domain_Knowledge/Rules of Engagement/3...
nlp_word,N2,dietary,reduced,RUF
nlp_tag,NNP,JJ,VBN,NNP
nlp_type,FAC,NaN,NaN,ORG
nlp_pofs,PROPN,ADJ,VERB,PROPN
is_probe,False,False,False,False
is_probe_probability,0.0,0.0,0.0,0.0


In [5]:

if nu.pickle_exists('deduped_lower_case_ners_df'): deduped_lower_case_ners_df = nu.load_object('deduped_lower_case_ners_df')
else:
    type_columns = ['ent_type', 'nlp_type', 'bert_entity']
    text_columns = ['ent_phrase', 'nlp_word', 'bert_word']
    
    # Get the person/date type mask
    type_values = ['PERSON', 'DATE', 'I-PER']
    base_mask_series = True
    for type_column in type_columns: base_mask_series &= ~domain_doc_ners_df[type_column].isin(type_values)
    
    # Get the null mask
    null_mask_series = False
    for text_column in text_columns: null_mask_series |= ~domain_doc_ners_df[text_column].isnull()
    
    # Loop through the lower-cased text strings and add them to the dataset
    mask_series = base_mask_series & null_mask_series
    non_named_df = domain_doc_ners_df[mask_series].drop(columns=['file_path', 'is_probe', 'is_probe_probability'])
    text_list = []
    for text_column in text_columns: text_list += non_named_df[text_column].map(lambda x: str(x).lower()).tolist()
    text_set = set(text_list)
    rows_list = []
    for text_str in text_set:
        mask_series = False
        for text_column in text_columns: mask_series |= non_named_df[text_column].map(lambda x: str(x).lower() == text_str)
        df = non_named_df[mask_series]
        mask_series = (df.apply(Series.nunique, axis='index') == 1)
        columns_list = df.columns[mask_series]
        row_dict = {'text_str': text_str}
        for cn in columns_list: row_dict[cn] = df.iloc[0][cn]
        rows_list.append(row_dict)
    deduped_lower_case_ners_df = DataFrame(rows_list)
    nu.store_objects(deduped_lower_case_ners_df=deduped_lower_case_ners_df)
    nu.save_data_frames(deduped_lower_case_ners_df=deduped_lower_case_ners_df)
print(deduped_lower_case_ners_df.shape)

(47270, 14)



## Anything that is not a name probably needs to be lower case before deduping

In [8]:

mask_series = (domain_doc_ners_df.bert_entity == 'I-PER')
df = domain_doc_ners_df[mask_series]
print(df.shape)
display(df.bert_word.unique().tolist())
df.sample(min(3, df.shape[0])).dropna(axis='columns', how='all').T

(288, 16)


['Remley', 'Loos', 'Jamie', 'iesberg', 'Michael', 'mley', 'Mosely', 'Montgomery', 'rendon', 'Keenan', 'John', 'tha', 'Papalski', 'Brit', 'Scarborough', 'S', 'Tripp', 'Shelia', 'Savell', 'Brendan', 'Fisher', 'Fruendt', 'man', 'Steven', 'avides', 'Walter', 'gle', 'Laura', 'ey', 'Otten', 'Jensen', 'Matthew', 'C', 'Danalieger', 'L', 'VanFosson', 'bas', 'gado', 'Male', 'gt', 'Tom', 'Adams', 'Twillige', 'Don', 'Parsons', 'Wanek', 'Levi', 'Tadlock', 'hauer', 'ylor', 'aa', 'Lorenzo', 'MICHAEL', 'REMY', 'Givens', 'F', 'Kane', 'Balfour', 'E', '.', 'ITHSICI', 'TOY', 'KIM', 'GELA', 'ARNELL', 'INDY', 'ULLEA', 'NEIL', 'GRUNBERG', 'ALTHEA', 'MAIN', 'ICKNELLAL', 'James', 'McNan', 'AMES', 'SCHWARTZ', 'GAN', 'MULLERNRAD', 'CHAEL', 'ST', 'BRUEGGEME', 'ES', 'MULLNRAD', 'A', 'Deuster', 'M', 'Scott', 'Andrew', 'Drew', 'Morgan', 'McClain', 'DEUSTER', 'O', 'TRAVIS', 'LUNASCOUMAN', 'OODSON', 'broise', 'ré', 'Jean', 'rrey', 'Pringle', 'Rush', 'Brocklesby', 'Letterman', 'Hammond', 'Sternberg', 'N', 'UPCHAK', 'AN

,87095,150711,87097
bert_word,ISAIAH,Philipp,MICHAEL
bert_entity,I-PER,I-PER,I-PER
bert_score,0.502805,0.999487,0.760972
bert_start,132.0,1449.0,161.0
bert_end,138.0,1460.0,168.0
file_path,../data/Domain_Knowledge/Fundamentals of Milit...,../data/Domain_Knowledge/Fundamentals of Milit...,../data/Domain_Knowledge/Fundamentals of Milit...
is_probe,False,False,False
is_probe_probability,0.0,0.0,0.0



### Value Counts

In [8]:

column_descriptions_df.sort_values('count_uniques')

,column_name,dtype,count_blanks,count_uniques,count_zeroes,has_dates,min_value,max_value,only_integers
0,is_probe,bool,0,2,1527921,False,False,True,True
8,bert_entity,object,1527216,5,0,False,I-LOC,I-PER,NaN
12,nlp_type,object,1362215,19,0,False,CARDINAL,WORK_OF_ART,NaN
13,nlp_pofs,object,88944,19,0,False,ADJ,X,NaN
15,ent_type,object,1443449,19,0,False,CARDINAL,WORK_OF_ART,NaN
11,nlp_tag,object,88944,51,0,False,$,``,NaN
9,file_path,object,0,79,0,False,../data/Domain_Knowledge/Fundamentals of Milit...,../data/Domain_Knowledge/soldiers-manual-68w.txt,NaN
7,bert_word,object,1527216,1011,0,True,&,“,NaN
2,bert_start,float64,1527216,1341,4,True,0.0,2309.0,False
3,bert_end,float64,1527216,1345,0,True,4.0,2318.0,False


In [14]:

domain_doc_ners_df.bert_entity.value_counts()

bert_entity
I-ORG     1589
I-LOC      362
I-MISC     340
I-PER      299
Name: count, dtype: int64

In [15]:

domain_doc_ners_df.nlp_type.value_counts()

nlp_type
ORG            61934
CARDINAL       25009
DATE           19601
PERSON         19057
GPE            12979
WORK_OF_ART     7528
LAW             3600
EVENT           3579
NORP            3310
PRODUCT         2147
TIME            1720
PERCENT         1572
ORDINAL         1471
FAC             1383
LOC             1359
QUANTITY        1025
MONEY            267
LANGUAGE          50
Name: count, dtype: int64

In [16]:

domain_doc_ners_df.nlp_pofs.value_counts()

nlp_pofs
NOUN     278639
PUNCT    176386
ADP      136987
PROPN    133600
SPACE    119534
VERB     111218
ADJ      107271
DET      104793
AUX       57001
CCONJ     46613
NUM       40600
PRON      36774
ADV       33315
PART      27316
SCONJ     18665
X          6081
SYM        5907
INTJ        162
Name: count, dtype: int64

In [17]:

domain_doc_ners_df.ent_type.value_counts()

ent_type
ORG            23417
CARDINAL       21897
PERSON         10920
DATE            9578
GPE             8443
NORP            3029
ORDINAL         1471
WORK_OF_ART     1448
LAW             1234
PRODUCT         1060
EVENT            835
LOC              697
PERCENT          668
TIME             664
FAC              464
QUANTITY         385
MONEY             97
LANGUAGE          50
Name: count, dtype: int64

In [18]:

domain_doc_ners_df.nlp_tag.value_counts()

nlp_tag
NN       192749
IN       151078
NNP      128294
_SP      119535
DT       108337
JJ       103544
NNS       86267
,         62956
.         62883
CC        46613
VB        42159
CD        40601
RB        36907
VBN       35267
VBZ       23434
VBG       19272
TO        17560
MD        17171
VBP       15887
VBD       14896
:         14371
PRP       13276
-RRB-     10839
-LRB-      9913
PRP$       8170
HYPH       7029
WDT        6038
SYM        5814
NNPS       5314
POS        3802
LS         3588
``         3443
WRB        3416
WP         3358
''         3273
JJR        2303
NFP        1682
ADD        1531
RBR        1458
EX         1452
JJS        1420
RP         1255
RBS         914
XX          632
PDT         405
FW          330
WP$         166
UH          163
$            93
AFX           4
Name: count, dtype: int64


## Also I think we should remove any dates, year, etc.

In [25]:

def describe_regexed_column(regexed_column, verbose=True):
    mask_series = (domain_doc_ners_df[regexed_column] == 'DATE')
    df = domain_doc_ners_df[mask_series]
    df1 = df.sample(min(4, df.shape[0])).dropna(axis='columns', how='all').T
    if verbose: display(df1)
    sample_columns_list = df1.index.tolist()
    if verbose: print(f'The {nu.conjunctify_nouns(sample_columns_list)} columns have data in them when the {regexed_column} column == "DATE"')

    # Check if the column contains any dates
    date_series = pd.to_datetime(df.nlp_word, errors='coerce')
    null_series = date_series[~date_series.notnull()]
    if verbose:
        if (null_series.shape[0] < date_series.shape[0]): print(f'The {regexed_column} column contains dates')
        else: print(f'The {regexed_column} column does not contain dates')

    # Display the descriptions of the sample columns
    if verbose: display(nu.get_column_descriptions(df, sample_columns_list).T)

In [26]:

search_regex = re.compile(r'^DATE$')
regexed_columns_list = nu.get_regexed_columns(domain_doc_ners_df, search_regex=search_regex)
print(regexed_columns_list)
df = nu.get_regexed_dataframe(domain_doc_ners_df, regexed_columns_list, search_regex=search_regex)
display(df.dropna(axis='columns', how='all').T)

['nlp_word', 'nlp_type', 'ent_type']


,1198556,27,28456
file_path,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/OSU Additions/17-13-t...,../data/Domain_Knowledge/OSU Additions/17-13-t...
nlp_word,DATE,MAY,NaN
nlp_tag,NNP,NNP,NaN
nlp_type,NaN,DATE,NaN
nlp_pofs,PROPN,PROPN,NaN
ent_phrase,NaN,NaN,MAY 17
ent_type,NaN,NaN,DATE
ent_start,NaN,NaN,67.0
ent_end,NaN,NaN,73.0
is_probe,False,False,False


In [27]:

describe_regexed_column('nlp_type')

,1421239,719724,215052,472429
file_path,../data/Domain_Knowledge/Rules of Engagement/A...,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Fundamentals of Milit...,../data/Domain_Knowledge/Fundamentals of Milit...
nlp_word,-,44,last,2009
nlp_tag,SYM,CD,JJ,CD
nlp_type,DATE,DATE,DATE,DATE
nlp_pofs,SYM,NUM,ADJ,NUM
is_probe,False,False,False,False
is_probe_probability,0.0,0.0,0.0,0.0


The file_path, nlp_word, nlp_tag, nlp_type, nlp_pofs, is_probe, and is_probe_probability columns have data in them when the nlp_type column == "DATE"
The nlp_type column contains dates


,0,1,2,3,4,5,6
column_name,is_probe,is_probe_probability,file_path,nlp_word,nlp_tag,nlp_type,nlp_pofs
dtype,bool,float64,object,object,object,object,object
count_blanks,0,0,0,0,0,0,0
count_uniques,1,1,76,1399,32,1,17
count_zeroes,19601,19601,0,0,0,0,0
has_dates,False,True,False,True,False,False,False
min_value,False,0.0,../data/Domain_Knowledge/Fundamentals of Milit...,\n,'',DATE,ADJ
max_value,False,0.0,../data/Domain_Knowledge/Tactical_Combat_Casua...,”1(Art,_SP,DATE,X
only_integers,True,True,NaN,NaN,NaN,NaN,NaN


In [28]:

describe_regexed_column('ent_type')

,1013168,1523392,703340,568195
file_path,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Rules of Engagement/A...,../data/Domain_Knowledge/Military Medical Ethi...,../data/Domain_Knowledge/Fundamentals of Milit...
ent_phrase,the final days,5,6 months,1951
ent_type,DATE,DATE,DATE,DATE
ent_start,89400.0,242702.0,161725.0,8479.0
ent_end,89414.0,242703.0,161733.0,8483.0
is_probe,False,False,False,False
is_probe_probability,0.002707,0.00274,0.002684,0.002721


The file_path, ent_phrase, ent_type, ent_start, ent_end, is_probe, and is_probe_probability columns have data in them when the ent_type column == "DATE"
The ent_type column does not contain dates


,0,1,2,3,4,5,6
column_name,is_probe,ent_start,ent_end,is_probe_probability,file_path,ent_phrase,ent_type
dtype,bool,float64,float64,float64,object,object,object
count_blanks,0,0,0,0,0,0,0
count_uniques,1,9363,9376,2902,76,3653,1
count_zeroes,9578,0,0,0,0,0,0
has_dates,False,True,True,True,False,True,False
min_value,False,45.0,53.0,0.002389,../data/Domain_Knowledge/Fundamentals of Milit...,+ 32,DATE
max_value,False,791001.0,791007.0,0.018902,../data/Domain_Knowledge/Tactical_Combat_Casua...,”1(Art 19,DATE
only_integers,True,True,True,False,NaN,NaN,NaN


In [29]:

# Get all the columns that have dates in them
mask_series = column_descriptions_df.has_dates
date_columns_list = column_descriptions_df[mask_series].column_name.tolist()
display(date_columns_list)

['bert_score', 'bert_start', 'bert_end', 'ent_start', 'ent_end', 'is_probe_probability', 'bert_word', 'nlp_word', 'ent_phrase']

In [32]:

mask_series = column_descriptions_df.column_name.isin(date_columns_list)
display(column_descriptions_df[mask_series].T)

,1,2,3,4,5,6,7,10,14
column_name,bert_score,bert_start,bert_end,ent_start,ent_end,is_probe_probability,bert_word,nlp_word,ent_phrase
dtype,float64,float64,float64,float64,float64,float64,object,object,object
count_blanks,1527216,1527216,1527216,1443452,1443452,0,1527216,88978,1443453
count_uniques,2582,1341,1345,68405,68225,24942,1011,47824,28437
count_zeroes,0,4,0,68,0,1443453,0,0,0
has_dates,True,True,True,True,True,True,True,True,True
min_value,0.310521,0.0,4.0,0.0,1.0,0.0,&,\n,#
max_value,0.999842,2309.0,2318.0,791001.0,791007.0,0.873549,“,﻿COMBAT,﻿COMBAT LIFESAVER
only_integers,False,False,False,False,False,False,NaN,NaN,NaN


In [48]:

def get_coerced_dates(column_name, verbose=True):
    mask_series = pd.to_datetime(domain_doc_ners_df[column_name], errors='coerce').notnull()
    df = domain_doc_ners_df[mask_series]
    display(sorted(df[column_name].unique()))
    display(df.sample(min(4, df.shape[0])).dropna(axis='columns', how='all').T)
    if (column_name == 'bert_word'): display(sorted(df.bert_entity.unique()))
    elif (column_name == 'nlp_word'): display(sorted([str(cv) for cv in df.nlp_type.unique()]))
    elif (column_name == 'ent_phrase'): display(sorted(df.ent_type.unique()))

In [ ]:

# Get the values in the bert_word column that can be coerced into dates
get_coerced_dates('bert_word')

In [ ]:

# Get the values in the nlp_word column that can be coerced into dates
get_coerced_dates('nlp_word')

In [ ]:

# Get the values in the ent_phrase column that can be coerced into dates
get_coerced_dates('ent_phrase')

In [52]:

type_columns = ['nlp_type', 'ent_type']
text_columns = ['nlp_word', 'ent_phrase']
for text_column, type_column in zip(text_columns, type_columns):
    mask_series = pd.to_datetime(domain_doc_ners_df[text_column], errors='coerce').notnull()
    domain_doc_ners_df.loc[mask_series, type_column] = 'DATE'
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)

Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv



----
### Maintenance

In [33]:

type_columns = ['ent_type', 'nlp_type', 'bert_entity']
text_columns = ['ent_phrase', 'nlp_word', 'bert_word']
columns_list = ['file_path'] + type_columns + text_columns
domain_doc_ners_df = domain_doc_ners_df.drop_duplicates(subset=columns_list, ignore_index=True)
print(domain_doc_ners_df.shape)
nu.save_data_frames(domain_doc_ners_df=domain_doc_ners_df)

(302127, 16)
Saving to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/csv/domain_doc_ners_df.csv
